In [ ]:
mu = [40., 50., 1000.]
sigma = [5., 2.5, 200.]
import numpy as np

In [ ]:
def obj(x, none_variable):
    sigma_y = x[0]
    w = x[1]
    m = x[2]
    return sigma_y*w - m

def grad_obj(x, none_variable):
    sigma_y = x[0]
    w = x[1]
    m = x[2]
    grad_g = [w, sigma_y, -1]
    return grad_g

In [ ]:
jacobian_xy = np.diag(stds)